In [9]:
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import torch
import math
import networkx as nx
from tqdm import tqdm
from PIL import Image
import json
import copy
from torch.utils.data import DataLoader, random_split

from expbasics.helper import get_attributions, get_model_etc, to_name
from expbasics.network import load_model, train_network, accuracy_per_class
from expbasics.biased_noisy_dataset import get_biased_loader, BiasedNoisyDataset

%reload_ext autoreload
%autoreload 2

In [14]:
BIASES = [0]
# BIASES = list(np.round(np.linspace(0, 1, 21), 3))
SEEDS = [14, 6, 1, 9, 5, 15, ]

NAME = "test"
rand_gen = torch.Generator().manual_seed(431)

BATCH_SIZE = 128
LEARNING_RATE = 0.001
EPOCHS = 2
STRENGTH = 0.5
IMAGE_PATH = "../dsprites-dataset/images"
accs = []
for bias in BIASES:
    for seed in SEEDS:
        ds = BiasedNoisyDataset(bias, STRENGTH, img_path=IMAGE_PATH)
        trainds, testds, _ = random_split(ds, [0.1, 0.01, 0.89], generator=rand_gen)
        train_loader = DataLoader(
            trainds, batch_size=BATCH_SIZE, shuffle=True, generator=rand_gen
        )
        test_loader = DataLoader(
            testds, batch_size=BATCH_SIZE, shuffle=True, generator=rand_gen
        )
        name = to_name(bias, seed)
        model = train_network(
            train_loader,
            bias,
            STRENGTH,
            NAME,
            BATCH_SIZE,
            load=False,
            retrain=False,
            learning_rate=LEARNING_RATE,
            epochs=EPOCHS,
            num_it=seed,
            seeded=True,
            disable=False,
        )

cpu
test_0_14.pickle
EPOCH 1:


385it [00:32, 11.82it/s, acc=80.2, batch=300, epoch=1, loss=0.43]


loss epoch: 0.43479172259569165
EPOCH 2:


385it [00:35, 10.97it/s, acc=99, batch=300, epoch=2, loss=0.03]  


loss epoch: 0.03294716692529619
best loss:  0.03294716692529619  last loss:  0.03294716692529619  best epoch:  model_1_0
cpu
test_0_6.pickle
EPOCH 1:


...

loss epoch: 0.04533252191729844
best loss:  0.04533252191729844  last loss:  0.04533252191729844  best epoch:  model_1_0
cpu
test_0_15.pickle
EPOCH 1:


385it [00:38,  9.98it/s, acc=59.7, batch=300, epoch=1, loss=0.65]


loss epoch: 0.6501215332746506
EPOCH 2:


385it [00:34, 11.07it/s, acc=62.9, batch=300, epoch=2, loss=0.58]

loss epoch: 0.5838315981626511
best loss:  0.5838315981626511  last loss:  0.5838315981626511  best epoch:  model_1_0


In [5]:
1 -(0.1+ 0.01)

0.89